In [52]:
import numpy as np

In [53]:
irr = '100011011'
mix_columns_matrix = ['02', '03', '01', '01',
                      '01', '02', '03', '01',
                      '01', '01', '02', '03',
                      '03', '01', '01', '02']

inverse_mix_columns_matrix = ['0e', '0b', '0d', '09',
                              '09', '0e', '0b', '0d',
                              '0d', '09', '0e', '0b',
                              '0b', '0d', '09', '0e']

In [54]:
def hex_bin(n):  #  converts between binary and hexadecimal
    hex_digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']
    bin_digits = [f'{a:0{4}b}' for a in range(16)]
    if n in hex_digits:
        return bin_digits[hex_digits.index(n)]
    if n in bin_digits:
        return hex_digits[bin_digits.index(n)]
    return 'invalid'

def dec_hex(n):  #  converts between decimal and hexadecimal
    hex_digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']
    dec_digits = [f'{a}' for a in range(16)]
    if n in hex_digits:
        return dec_digits[hex_digits.index(n)]
    if n in dec_digits:
        return hex_digits[dec_digits.index(n)]
    return 'invalid'

print(hex_bin('0101'))
print(dec_hex('d'))

5
13


In [55]:
def xor(a, b):
    c = ''
    for i in range(len(a)):
        if a[i] == b[i]:
            c += '0'
        else:
            c += '1'
    return c

In [56]:
def mult_2(pol, irr):  #  multiplication by polynomial 00000010 in GF(256) using an irreducible polynomial
    pol = pol + '0'
    if pol[0] == '1':  #  if we have an X^8 term we reduce modulo the irreducible polynomial
        pol = xor(pol, irr)
    pol = pol[1:]  #  slice back to 8 binary digits
    return pol

print(mult_2('10000111', '100011011'))

00010101


In [57]:
#  hexadecimal xor list
hex_xor_list = []
for d1 in range(16):
    for d2 in range(16):
        hex_xor_list.append(hex_bin(xor(f'{d1:0{4}b}', f'{d2:0{4}b}')))

#  build list of results of the operation 02 * h in GF(256) for hex numbers h from 00 to FF using stated irreducible polynomial for faster calculation

two_pol = []
for d1 in range(16):
    for d2 in range(16):
        mult = mult_2(f'{d1:0{4}b}' + f'{d2:0{4}b}', irr)
        two_pol.append(hex_bin(mult[:4]) + hex_bin(mult[4:]))

print('xor list:\n', hex_xor_list)
print('\nmult by 02x list:\n', two_pol)
      

xor list:
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', 'b', 'a', 'd', 'c', 'f', 'e', '2', '3', '0', '1', '6', '7', '4', '5', 'a', 'b', '8', '9', 'e', 'f', 'c', 'd', '3', '2', '1', '0', '7', '6', '5', '4', 'b', 'a', '9', '8', 'f', 'e', 'd', 'c', '4', '5', '6', '7', '0', '1', '2', '3', 'c', 'd', 'e', 'f', '8', '9', 'a', 'b', '5', '4', '7', '6', '1', '0', '3', '2', 'd', 'c', 'f', 'e', '9', '8', 'b', 'a', '6', '7', '4', '5', '2', '3', '0', '1', 'e', 'f', 'c', 'd', 'a', 'b', '8', '9', '7', '6', '5', '4', '3', '2', '1', '0', 'f', 'e', 'd', 'c', 'b', 'a', '9', '8', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', '0', '1', '2', '3', '4', '5', '6', '7', '9', '8', 'b', 'a', 'd', 'c', 'f', 'e', '1', '0', '3', '2', '5', '4', '7', '6', 'a', 'b', '8', '9', 'e', 'f', 'c', 'd', '2', '3', '0', '1', '6', '7', '4', '5', 'b', 'a', '9', '8', 'f', 'e', 'd', 'c', '3', '2', '1', '0', '7', '6', '5', '4', 'c', 'd', 'e', 'f', '8', '9'

In [58]:
def f_hex_xor(x, y):
    return hex_xor_list[int(dec_hex(x[0])) * 16 + int(dec_hex(y[0]))] + hex_xor_list[int(dec_hex(x[1])) * 16 + int(dec_hex(y[1]))]

def f_mult2(pol_hex):  #  multiplication by 02
        return two_pol[int(dec_hex(pol_hex[0])) * 16 + int(dec_hex(pol_hex[1]))]

def gen_mult_hex(mult, pol_hex):  #  !!!!! use this to generate a precaluclated table for 03, 09, 0b, 0d, 0e for both forward and inverse mix columns
    if mult == '01':
        return pol_hex
    if mult == '02':
        return f_mult2(pol_hex)
    if mult == '03':
        return f_hex_xor(f_mult2(pol_hex), pol_hex)  # 02 xor 01 = 03
    if mult == '09':
        return f_hex_xor(f_mult2(f_mult2(f_mult2(pol_hex))), pol_hex)  #  (02 x 02 x 02) xor 01 = 09 or (X * X * X) + 1 = X^3 + 1
    if mult == '0b':
        return f_hex_xor(gen_mult_hex('09', pol_hex), f_mult2(pol_hex))  # 09 xor 02 = 0b or (X^3 + 1) + X = X^3 + X + 1
    if mult == '0d':
        return f_hex_xor(gen_mult_hex('09', pol_hex), f_mult2(f_mult2(pol_hex))) # 09 xor (02 x 02) = 0d or (X^3 + 1) + (X * X) = X^3 + X^2 + 1
    if mult == '0e':
        return f_hex_xor(f_hex_xor(f_mult2(f_mult2(f_mult2(pol_hex))), f_mult2(f_mult2(pol_hex))), f_mult2(pol_hex))
        # (02 x 02 x 02) xor (02 x 02) xor 02 = 0e or (X * X * X) + (X * X) + X = X^3 + X^2 + X
    
    


print(f_hex_xor('a1', '32')) # 93
print(f_hex_xor('af', '00')) # af
print(gen_mult_hex('01', 'fe')) # fe
print(gen_mult_hex('02', '87')) # 15
print(gen_mult_hex('03', '6e')) # b2
print(gen_mult_hex('09', 'ae')) # a9
print(gen_mult_hex('0b', '02')) # 16
print(gen_mult_hex('0d', '11')) # dd
print(gen_mult_hex('0e', 'cc')) # a4

#  simplify functions add xor of hex function using tables

93
af
fe
15
b2
a9
16
dd
a4


In [59]:
#  mix columns step function, receiving mixing matrix and input block use inverse for decryption
def mix_columns(mix_matrix, block_matrix):  #  even though labelled as "matrices", these are unidimensional lists
    mixed = []
    for j in range(16):
            element = '00'
            for k in range(4):
                element = f_hex_xor(element, gen_mult_hex(mix_matrix[(j//4) * 4 + k], block_matrix[j % 4 + 4 * k]))
            mixed.append(element)
    return mixed

test = ['47', '40', 'a3', '4c',
        '37', 'd4', '70', '9f',
        '94', 'e4', '3a', '42',
        'ed', 'a5', 'a6', 'bc']

test2 = ['87', 'f2', '4d', '97',
        '6e', '4c', '90', 'ec',
        '46', 'e7', '4a', 'c3',
        'a6', '8c', 'd8', '95']

print(mix_columns(inverse_mix_columns_matrix, ['ae', 'a6', '51', 'dc', '63', '61', 'ab', '13', '6f', 'b3', '56', '88', 'ab', 'a1', 'e3', '6a']))
print(mix_columns(inverse_mix_columns_matrix, ['47', '40', 'a3', '4c', '37', 'd4', '70', '9f', '94', 'e4', '3a', '42', 'ed', 'a5', 'a6', 'bc']))
print(mix_columns(mix_columns_matrix, test))  # ['ae', 'a6', '51', 'dc', '63', '61', 'ab', '13', '6f', 'b3', '56', '88', 'ab', 'a1', 'e3', '6a']
print(mix_columns(mix_columns_matrix, test2)) # ['47', '40', 'a3', '4c', '37', 'd4', '70', '9f', '94', 'e4', '3a', '42', 'ed', 'a5', 'a6', 'bc']

['47', '40', 'a3', '4c', '37', 'd4', '70', '9f', '94', 'e4', '3a', '42', 'ed', 'a5', 'a6', 'bc']
['87', 'f2', '4d', '97', '6e', '4c', '90', 'ec', '46', 'e7', '4a', 'c3', 'a6', '8c', 'd8', '95']
['ae', 'a6', '51', 'dc', '63', '61', 'ab', '13', '6f', 'b3', '56', '88', 'ab', 'a1', 'e3', '6a']
['47', '40', 'a3', '4c', '37', 'd4', '70', '9f', '94', 'e4', '3a', '42', 'ed', 'a5', 'a6', 'bc']
